# Face Recognition Model 3

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import keras

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop

In [1]:
import os
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: c:\Users\makpe\OneDrive\Documents\Data Scientist\Deep Learning\Final_Project_DL_Oct_23


## EDA

In [2]:
pic_size = 48

folder_path = current_directory + "/face_recognition_dataset/"

class_names = []

for expression in os.listdir(folder_path+"Testing/Testing/"):
    class_names.append(expression)
    print(str(len(os.listdir(folder_path+"Testing/Testing/"+expression)))+" "+expression+" images")
    pass

958 Angry images
1024 Fear images
1774 Happy images
1233 Neutral images
1247 Sad images
831 Suprise images


## Load Class Function

In [3]:
def load_class(class_names, index, num_examples, mode):
    images = []
    class_name = class_names[index]
    
    TRAIN_PATH = folder_path + "Training/Training/" + class_name
    TEST_PATH = folder_path + "Testing/Testing/" + class_name
    PATH = ''
    if mode == 0:
        PATH = TRAIN_PATH
    else:
        PATH = TEST_PATH
    
    img_names = os.listdir(PATH)
    img_names = np.random.RandomState(seed=69).permutation(img_names)
    
    for i in range(num_examples):
        img = io.imread(PATH + "/" + img_names[i])
        img = img.reshape([48, 48, 1])
        images.append(img)

    labels = np.empty(num_examples)
    labels.fill(index)
    
    return np.array(images), keras.utils.to_categorical(labels, len(class_names), dtype=int)

In [28]:
from skimage import io

data_train = []
label_train = []
for i in range(len(class_names)):
    data_part, label_part = load_class(class_names, i, 3000, 0)
    data_train.extend(data_part)
    label_train.extend(label_part)
    
X_train = np.array(data_train)
y_train = np.array(label_train)

In [30]:
data_test = []
label_test = []
for i in range(len(class_names)):
    data_part, label_part = load_class(class_names, i, 800, 1)
    data_test.extend(data_part)
    label_test.extend(label_part)
    
X_test = np.array(data_test)
y_test = np.array(label_test)

## Image Preprocessing

In [18]:
batch_size = 128

datagen_train = ImageDataGenerator(rescale= 1/255., 
                             horizontal_flip= True, 
                             rotation_range=30, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1, 
                             shear_range=0.2,
                             zoom_range=0.2, 
                             fill_mode='nearest')

datagen_test = ImageDataGenerator(rescale= 1/255.)

train_generator = datagen_train.flow(X_train, y_train, batch_size = 64, shuffle = True, seed = 69)

val_generator = datagen_test.flow(X_test, y_test, batch_size = 64, shuffle = True, seed = 69)

## Build the Model

In [10]:
from keras.optimizers import Adam,SGD,RMSprop

no_of_classes = 6

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))



opt = Adam(lr = 0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (Batch  (None, 48, 48, 64)        256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

## Callback List

In [36]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./face_rec.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0.00005,
                          patience=8,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 48

optimizer=RMSprop(learning_rate=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

## Fit the Model

In [39]:
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data = val_generator,
                    callbacks=callbacks_list
                    )

Epoch 1/48
40/40 [==============================] - ETA: 0s - loss: 1.0301 - accuracy: 0.5893
Epoch 1: val_accuracy did not improve from 0.57879
40/40 [==============================] - 30s 757ms/step - loss: 1.0301 - accuracy: 0.5893 - val_loss: 1.1354 - val_accuracy: 0.5682 - lr: 3.2000e-07
Epoch 2/48
40/40 [==============================] - ETA: 0s - loss: 1.0396 - accuracy: 0.5968
Epoch 2: val_accuracy did not improve from 0.57879
40/40 [==============================] - 31s 780ms/step - loss: 1.0396 - accuracy: 0.5968 - val_loss: 1.1360 - val_accuracy: 0.5697 - lr: 3.2000e-07
Epoch 3/48
40/40 [==============================] - ETA: 0s - loss: 1.0464 - accuracy: 0.5897
Epoch 3: val_accuracy did not improve from 0.57879
40/40 [==============================] - 32s 805ms/step - loss: 1.0464 - accuracy: 0.5897 - val_loss: 1.1366 - val_accuracy: 0.5697 - lr: 3.2000e-07
Epoch 4/48
40/40 [==============================] - ETA: 0s - loss: 1.0210 - accuracy: 0.6036
Epoch 4: val_accuracy di